# Day 5: Hydrothermal Venture
https://adventofcode.com/2021/day/5

In [2]:
use std::fs;

let input_lines = fs::read_to_string("input/day05.txt")
    .expect("Could not read file.")
    .lines()
    .map(|line| line.to_string())
    .collect::<Vec<String>>();

We will use regular expressions to parse the input.

In [3]:
:dep regex = "1.5.4"

In [4]:
use regex::Regex;

In [5]:
type Point = (i32, i32);
type Line = (Point, Point);

In [6]:
fn parse_line(line: &String) -> Line {
    let RE: Regex = Regex::new(r"(\d+),(\d+) -> (\d+),(\d+)").unwrap();

    let numbers = RE.captures(line).unwrap().iter()
        .skip(1)  // the first element of the iterator is the full matched string
        .map(|n| n.unwrap().as_str().parse().unwrap())
        .collect::<Vec<i32>>();
    
    // slice patterns: https://stackoverflow.com/questions/32324645/how-can-i-unpack-destructure-elements-from-a-vector
    
    if let [x1, y1, x2, y2] = &numbers[..] {
        ((*x1, *y1), (*x2, *y2))
    } else {
        panic!("Could not parse line")
    }
}

In [7]:
let parsed_input_lines: Vec<Line> = input_lines.iter().map(|line| parse_line(line)).collect();

Verify that parsing works:

In [8]:
input_lines[0..3]

["72,504 -> 422,154", "877,851 -> 680,654", "447,989 -> 517,989"]

In [9]:
parsed_input_lines[0..3]

[((72, 504), (422, 154)), ((877, 851), (680, 654)), ((447, 989), (517, 989))]

For part 1, only horizontal and vertical lines are interesting.

In [10]:
fn horizontal_or_vertical(line: &Line) -> bool {
    let ((x1, y1), (x2, y2)) = *line;
    
    match (x2 - x1, y2 - y1) {
        (_, 0) => true,
        (0, _) => true,
        _ => false
    }
    
}

Determine all points covered by a line. Note that all lines are horizontal, vertical, or diagonal with an angle of 45 degress.

In [11]:
use std::cmp::{min, max};

fn covered_points(line: &Line) -> Vec<Point> {
    let ((x1, y1), (x2, y2)) = *line;
    
    match (x2 - x1, y2 - y1) {
        (_, 0) => (min(x1, x2)..=max(x1, x2)).map( |x| (x, y1)).collect(),
        (0, _) => (min(y1, y2)..=max(y1, y2)).map( |y| (x1, y)).collect(),
        (dx, dy) => {
            let sx = dx.signum();
            let sy = dy.signum();
            
            (0..=dx.abs()).map(|i| (x1 + i * sx, y1 + i * sy)).collect()
        }
    }
}

We will use `itertools` for counting points with overlapping lines.

In [12]:
:dep itertools = "0.10.3"

In [13]:
use itertools::Itertools;

In [14]:
fn count_points_with_overlapping_lines(lines: &Vec<Line>, filter: &dyn Fn(&Line) -> bool) -> usize {
    lines.iter()
        .filter(|line| filter(line))
        .flat_map(|line| covered_points(line))
        .sorted()
        .group_by(|&point| point)
        .into_iter()
        .map(|(_point, group)| group.count())
        .filter(|&count| count > 1)
        .count()
}

## Part 1: only horizontal and vertical lines

In [15]:
count_points_with_overlapping_lines(&parsed_input_lines, &horizontal_or_vertical)

6666

## Part 2: all lines

In [16]:
count_points_with_overlapping_lines(&parsed_input_lines, &|_line| true)

19081